In [1]:
import os
import json
import pandas as pd
from urllib.request import urlretrieve
from urllib.error import HTTPError

In [2]:
root = 'https://raw.githubusercontent.com/TimeWz667/'
pattern_targets = root + 'IndTB_prep/V1.1-warroom/serve/WarRoom/{:s}/targets.csv'
pattern_care = root + 'IndTB_underreport/v0.3/docs/pars/pars_{:s}.json'

In [5]:
def retrieve_data(k, location):
    meta = dict(Name=location['Name'])

    if len(chain) > 1:
        chn = [
            (chain[0], f'State_{chain[0]}'),
            (chain[-2], f'Region_{chain[-2]}'),
            ('India', 'India')
        ]
    else:
        chn = [('India', 'India')]

    for loc, file in chn:
        try:
            path = pattern_targets.format(file)
            urlretrieve(path, f'data/{k}/targets.csv')
        except HTTPError:
            pass
        else:
            meta['Source_Targets'] = {
                'Location': loc,
                'Url': path
            }
            break

    if len(chain) > 1:
        chn = [
            (chain[0], f's_{chain[0]}'),
            (chain[-2], f'r_{chain[-2]}'),
            ('India', 'overall')
        ]
    else:
        chn = [('India', 'overall')]

    for loc, file in chn:
        try:
            path = pattern_care.format(file)
            urlretrieve(path, f'data/{k}/pars_under.json')
        except HTTPError:
            pass
        else:
            meta['Source_Care'] = {
                'Location': loc,
                'Url': path
            }
            break

    with open(f'data/{k}/meta.json', 'w') as f:
        json.dump(meta, f)

In [6]:
locations = pd.read_csv('data/locations.csv')
locations = locations.set_index('Location')
locations = {i: dict(row) for i, row in locations.iterrows()}

In [7]:
for k, location in locations.items():
    chain = location['Chain'].split(':')
    os.makedirs(f'data/{k}', exist_ok=True)
    retrieve_data(k, location)
